In [116]:
# Importación de librerías
import pandas as pd
import plotly.express as px
import unidecode
import os

# Ruta del archivo Excel
filePath = '../sdei-annual-pm2-5/sdei-annual-pm2-5-concentrations-countries-urban-areas-v1-1998-2016-xlsx.xlsx'

# Verificar si el archivo existe
if not os.path.exists(filePath):
    print(f"No se encontró el archivo: {filePath}")
    exit()

# Cargar el archivo Excel
excelData = pd.ExcelFile(filePath)

# Información de las hojas a cargar y los detalles de procesamiento
sheet_info = {
    'Country PM2.5 Exceedance': {
        'filter_prefix': 'PMEXDC_',
        'country_column': 'COUNTRY'
    },
    'Urban PM2.5 Exposure': {
        'filter_prefix': 'AVPMU_',
        'country_column': 'COUNTRYENG',
        'city_column': 'STNDRDNAME'
    }
}

# Función para normalizar nombres (países y ciudades)
def normalize_name(name):
    # Convertir a string por si hay valores nulos
    name = str(name)
    # Eliminar tildes y caracteres especiales
    name_without_accents = unidecode.unidecode(name)
    # Convertir a minúsculas y luego capitalizar
    name_formatted = name_without_accents.lower().capitalize()
    return name_formatted

# Función para leer y procesar cada hoja
def load_and_process_sheet(excel_data, sheet_name, filter_prefix, country_column, city_column=None):
    # Cargar la hoja
    df = excel_data.parse(sheet_name)
    
    # Filtrar las columnas que empiezan con el prefijo
    data_columns = [col for col in df.columns if col.startswith(filter_prefix)]
    
    # Renombrar las columnas eliminando el prefijo y espacios
    renamed_columns = {col: col.replace(filter_prefix, '').strip() for col in data_columns}
    df.rename(columns=renamed_columns, inplace=True)
    
    # Crear lista de columnas filtradas (sin prefijos)
    key_columns = [country_column]
    if city_column:
        key_columns.append(city_column)
    filtered_columns = key_columns + list(renamed_columns.values())
    
    # Seleccionar las columnas filtradas
    df = df[filtered_columns]
    
    # Normalizar los nombres de los países y ciudades
    df[country_column] = df[country_column].apply(normalize_name)
    if city_column:
        df[city_column] = df[city_column].apply(normalize_name)
    
    # Renombrar las columnas de país y ciudad para consistencia
    df = df.rename(columns={country_column: 'COUNTRY'})
    if city_column:
        df = df.rename(columns={city_column: 'CITY'})
    
    # Agregar una columna para indicar el indicador (nombre de la hoja)
    df['Indicator'] = sheet_name
    
    # Convertir a formato largo
    id_vars = ['COUNTRY', 'Indicator']
    if city_column:
        id_vars.insert(1, 'CITY')  # Insertar CITY después de COUNTRY
    df_long = pd.melt(df, id_vars=id_vars, var_name='Year', value_name='Value')
    
    # Convertir 'Year' a entero
    df_long['Year'] = df_long['Year'].astype(int)
    
    # Eliminar filas con valores nulos en 'Value'
    df_long = df_long.dropna(subset=['Value'])
    
    return df_long

# Procesar y combinar las hojas seleccionadas
df_list = []
for sheet_name, info in sheet_info.items():
    df_sheet = load_and_process_sheet(
        excel_data=excelData,
        sheet_name=sheet_name,
        filter_prefix=info['filter_prefix'],
        country_column=info['country_column'],
        city_column=info.get('city_column')  # Puede ser None si no está definido
    )
    df_list.append(df_sheet)

# Combinar las hojas en un solo DataFrame
df_combined = pd.concat(df_list, ignore_index=True)

# Función para filtrar datos
def filter_data(df, indicator=None, countries=None, cities=None, start_year=None, end_year=None):
    if indicator is not None:
        df = df[df['Indicator'] == indicator]
    if countries is not None:
        # Normalizar los nombres de los países en el filtro
        countries_normalized = [normalize_name(country) for country in countries]
        df = df[df['COUNTRY'].isin(countries_normalized)]
    if cities is not None and 'CITY' in df.columns:
        # Normalizar los nombres de las ciudades en el filtro
        cities_normalized = [normalize_name(city) for city in cities]
        df = df[df['CITY'].isin(cities_normalized)]
    if start_year is not None:
        df = df[df['Year'] >= start_year]
    if end_year is not None:
        df = df[df['Year'] <= end_year]
    return df

# Función para graficar histograma
def plot_histogram(df, title='Histograma'):
    fig = px.histogram(df, x='Value', nbins=30, title=title)
    fig.update_layout(xaxis_title='Valor', yaxis_title='Frecuencia')
    fig.show()

# Función para graficar mapa
def plot_map(df, year, title='Mapa'):
    # Filtrar por el año especificado
    df_year = df[df['Year'] == year].copy()
    
    # Asegurarnos de que 'Value' es numérico
    df_year['Value'] = pd.to_numeric(df_year['Value'], errors='coerce')
    
    # Eliminar filas con 'Value' nulo
    df_year = df_year.dropna(subset=['Value'])
    
    # Verificar si hay datos de ciudad
    if 'CITY' in df_year.columns:
        # Agrupar por país y calcular la media de 'Value'
        df_grouped = df_year.groupby('COUNTRY')['Value'].mean().reset_index()
    else:
        df_grouped = df_year.groupby('COUNTRY')['Value'].mean().reset_index()
    
    fig = px.choropleth(
        df_grouped,
        locations='COUNTRY',
        locationmode='country names',
        color='Value',
        hover_name='COUNTRY',
        color_continuous_scale=px.colors.sequential.Plasma,
        title=title
    )
    fig.update_geos(showframe=False, projection_type='equirectangular')
    fig.update_layout(legend_title_text='Valor')
    fig.show()


/home/juan27/.local/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning:

Cannot parse header or footer so it will be ignored



In [114]:
df_combined

,COUNTRY,Indicator,Year,Value,CITY
0,Afghanistan,Country PM2.5 Exceedance,2008,13.936634,NaN
1,Albania,Country PM2.5 Exceedance,2008,5.387369,NaN
2,Algeria,Country PM2.5 Exceedance,2008,0.586799,NaN
3,American samoa,Country PM2.5 Exceedance,2008,0.000000,NaN
4,Andorra,Country PM2.5 Exceedance,2008,0.000000,NaN
...,...,...,...,...,...
637614,China,Urban PM2.5 Exposure,2016,13.316667,Zhizhong
637615,China,Urban PM2.5 Exposure,2016,15.050000,Foluo
637616,China,Urban PM2.5 Exposure,2016,14.730000,Huangliu
637617,China,Urban PM2.5 Exposure,2016,14.594737,Chongpo


In [118]:
def plot_city_data(df, country, indicator, year):
    # Normalizar el nombre del país
    country_normalized = normalize_name(country)
    
    # Filtrar los datos según el país, indicador y año
    df_filtered = df[
        (df['COUNTRY'] == country_normalized) &
        (df['Indicator'] == indicator) &
        (df['Year'] == year)
    ].copy()
    
    # Verificar si la columna 'CITY' existe en el dataframe
    if 'CITY' not in df_filtered.columns:
        print(f"No hay datos de ciudad para el indicador '{indicator}'.")
        return
    
    # Asegurarnos de que 'Value' es numérico
    df_filtered['Value'] = pd.to_numeric(df_filtered['Value'], errors='coerce')
    df_filtered = df_filtered.dropna(subset=['Value'])
    
    # Verificar si hay datos después del filtrado
    if df_filtered.empty:
        print("No hay datos disponibles para los filtros especificados.")
        return
    
    # Crear el gráfico
    fig = px.bar(
        df_filtered,
        x='CITY',
        y='Value',
        title=f"{indicator} en {country_normalized} en {year} por Ciudad",
        labels={'Value': 'Valor', 'CITY': 'Ciudad'}
    )
    fig.update_layout(xaxis_tickangle=-45)
    fig.show()


In [119]:
# Parámetros
df = df_combined  # El dataframe combinado
country = 'Argentina'
indicator = 'Urban PM2.5 Exposure'
year = 2015

# Llamada a la función
plot_city_data(df, country, indicator, year)


In [115]:
# Ejemplo 1: Filtrar datos y graficar un histograma para ciudades específicas
indicator = 'Urban PM2.5 Exposure'  # Nombre exacto de la hoja
countries = ['Argentina', 'Brasil']
#cities = ['Buenos Aires', 'Rio De Janeiro']
start_year = 2010
end_year = 2015

# Filtrar datos
filtered_df = filter_data(
    df_combined,
    indicator=indicator,
    countries=countries,
    #cities=cities,
    start_year=start_year,
    end_year=end_year
)

# Verificar si el DataFrame no está vacío
if not filtered_df.empty:
    # Graficar histograma
    plot_histogram(filtered_df, title='Exposición Urbana al PM2.5 (2010-2015) en Ciudades Seleccionadas')
else:
    print("No hay datos para los filtros seleccionados.")


In [117]:
# Ejemplo 2: Graficar un mapa para un año específico
indicator = 'Country PM2.5 Exceedance'
year = 2015

# Filtrar datos
filtered_df = filter_data(
    df_combined,
    indicator=indicator,
    start_year=year,
    end_year=year
)

# Verificar si el DataFrame no está vacío
if not filtered_df.empty:
    # Graficar mapa
    plot_map(filtered_df, year=year, title='Exceso de PM2.5 por País en 2015')
else:
    print("No hay datos para los filtros seleccionados.")